In [ ]:
import pandas as pd 
import numpy as np 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding

In [ ]:
df=pd.read_csv('train.csv')

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df=df.dropna()

In [ ]:
x=df.iloc[:,:-1]
y=df.iloc[:,-1:]

In [ ]:
x.head(2)

## Text Pre proceesing

In [ ]:
message=df.copy()

In [ ]:
message.reset_index(inplace=True)

In [ ]:
message.head()

In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
import re
from nltk.stem import PorterStemmer
ps=PorterStemmer()
data=[]
for i in range(len(message)):
    review=re.sub('[^a-zA-Z]',' ',message['title'][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
    review=' '.join(review)
    data.append(review)


In [ ]:
data[0]

In [ ]:
from tensorflow.keras.preprocessing.text import one_hot
voc_size=5000
one_hot=[one_hot(words,voc_size) for words in data]

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded_data=pad_sequences(one_hot,padding='pre',maxlen=20)

In [ ]:
padded_data

In [ ]:
model=Sequential([
    Embedding(voc_size,40,input_length=20),
    LSTM(100),
    Dense(1,activation='sigmoid')

]
)

In [ ]:
model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
x=np.array(padded_data)
y=np.array(y)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=5)

In [56]:
y_pred=model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

189/189 [==============================] - 0s 2ms/step


In [57]:
from sklearn.metrics import accuracy_score,confusion_matrix
acc=accuracy_score(y_test,y_pred_binary)
print("Accuracy score is :{}".format(acc))

Accuracy score is :0.9148301574150787


In [58]:
y_pred = model.predict(X_test)
y_pred_labels = ["Fake" if pred < 0.5 else "Real" for pred in y_pred]

# Display first 10 predictions
for i, label in enumerate(y_pred_labels[:10]):  
    print(f"Prediction {i+1}: {label}")


189/189 [==============================] - 0s 2ms/step
Prediction 1: Real
Prediction 2: Fake
Prediction 3: Fake
Prediction 4: Real
Prediction 5: Real
Prediction 6: Real
Prediction 7: Fake
Prediction 8: Real
Prediction 9: Real
Prediction 10: Real


In [64]:
y_pred = model.predict(X_test)

# Convert predictions to labels
y_pred_labels = ["Fake" if pred < 0.5 else "Real" for pred in y_pred]

# Display text with predictions
for i in range(10):  # Show first 10 predictions
    print(f"Text: {X_test[i]}")  # Ensure this contains actual text
    print(f"Prediction: {y_pred_labels[i]}\n")


189/189 [==============================] - 0s 2ms/step
Text: [   0    0    0    0    0    0    0    0    0    0    0    0    0 2152
 3542 4185 4024 4179  872 2133]
Prediction: Real

Text: [   0    0    0    0    0    0    0    0    0    0    0 2257 3526 2749
 2489 4775 2782 4185  430 2646]
Prediction: Fake

Text: [   0    0    0    0    0    0    0    0    0    0 2407  588 4159  241
 4953  469 2788 4185  430 2646]
Prediction: Fake

Text: [   0    0    0    0    0    0    0    0    0    0    0    0    0 3972
 3847 1110  726 3557 4177 1356]
Prediction: Real

Text: [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
 3302 4526 1794 4306  845 2133]
Prediction: Real

Text: [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 2283 3402 4416 3984 4631]
Prediction: Real

Text: [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
 2741 2748 1254 1623  359 3026]
Prediction: Fake

Text: [   0    0    0    0    0    0    0    0    0    0